In [1]:
pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import faiss

import random
from sentence_transformers import SentenceTransformer

warnings.filterwarnings(action='ignore')

books = pd.read_csv('/content/drive/MyDrive/books_mod_fin', encoding='utf-8')
books = books.replace({np.nan: 'none'})


books_df_nax=books.dropna(axis='rows')


tfidf = TfidfVectorizer(stop_words='english')



def find_sim_books(U):
    def userbooks(A):
        x=books_df_nax['isbn'].tolist()
        l=len(A)
        BD=[]
        for i in range(l):
            if A[i] in x:
              BD.append(A[i])
        l2=len(BD)
        n=random.randrange(0,l2)
        return BD[n]

    # 사용자의 isbn 데이터를 담은 책 리스트를 input으로 받아 그 리스트 중 하나를 반환하는 함수
    # B = 사용자 데이터에서 랜덤 추출한 책의 isbn 
    B=userbooks(U) 


    def sim_idx_with_A(df,isbn):
        k=df[df['isbn']==isbn]['kdc'].item()
        print(k)
        n=int(df[df['isbn']==isbn]['des_cluster'])
        if n==280:
          m=int(df[df['isbn']==isbn]['tit_cluster'])
          same_clu_books_df=df[df['tit_cluster']==m]

        else: 
          same_clu_books_df=df[df['des_cluster']==n]

        sen = same_clu_books_df['content']

        print(len(sen))
        kdc_same_vect=[0 for i in range(len(sen))]
        for i in range(len(sen)):
          if same_clu_books_df[i:i+1]['kdc'].item()==k:
            kdc_same_vect[i]=0.05
        print(kdc_same_vect)
        


        # 1번 후보 : sentence transformer 유사도 구하기
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        sen_array = sen.to_numpy()
        sen_embeddings = model.encode(sen_array)
        clu_des_sim=cosine_similarity(sen_embeddings,sen_embeddings)

        # kdc 가 같으면 보너스 점수 주기 
    
        # 2번 후보 : 단순 tfidf 활용한 유사도 구하기
        # clu_tfidf = tfidf.fit_transform(sen)
        # clu_des_sim=cosine_similarity(clu_tfidf,clu_tfidf)


        idx=np.where(same_clu_books_df['isbn']==B)[0][0]
        books_sim_vect=clu_des_sim[idx:idx+1]
        sim_vect_plus_w=books_sim_vect+kdc_same_vect
        books_des_sim_idx = sim_vect_plus_w.argsort()[::-1]


        return books_des_sim_idx

    # 사용자 책 리스트에서 랜덤 추출한 책 기준 유사한 책 인덱스 리스트 반환
    sim_books_idx=sim_idx_with_A(books_df_nax,B)

    top_n=20
    top_sim_idx=sim_books_idx[0][:top_n]
    top_sim_idx=top_sim_idx.reshape(-1,)

    sim_books=books_df_nax.iloc[top_sim_idx]
    # 데이터프레임이 content, thumbnail column을 포함한다면
    outputs=sim_books[['isbn','title','author','content','thumbnail','publisher','kdc','price']]
    # outputs=sim_books[['title','author','img_url','publisher','kdc_class_no','price']]
    outputs_dic=outputs.to_dict('records')

    return outputs_dic

In [18]:
find_sim_books([9788990008343])

233
1529
[0, 0.05, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[{'isbn': 9788997348756,
  'title': '카페 인테리어 싸게 하기',
  'author': '이민 (지은이)',
  'content': '『카페 인테리어 싸게 하기』는 홍대의 ‘에티오피아’, 효창동 마다가스카르’, 금호동 ‘커피 브레이크 위드 새미’, 당산동 ‘커피 체리’, 삼성동 ‘카페 네프’ 등 인테리어 공사를 직접한 저자의 카페 인테리어 싸게 직접 하는 방법과 현장 사진, 노하우를 소개하고 있다.',
  'thumbnail': 'https://image.aladin.co.kr/product/10630/42/cover/e899734824_2.jpg',
  'publisher': '푸른영토',
  'kdc': '619.5',
  'price': '8900.0'},
 {'isbn': 9788991307520,
  'title': '김현석의 전공일반사회',
  'author': '김현석 (지은이)',
  'content': "중등임용고시 대비 전공일반사회 수험서. 이 책은 7차 교육과정에 맞춘 정치, 사회문화, 법, 경제의 방대한 양을 한 권에 압축 정리하여 수험생들이 효과적으로 학습할 수 있도록 하였다. 특히 중요사항을 표로 보기 좋게 정리하고, '참고'를 통해 놓치기 쉬운 부분을 짚어 주어 학습효과를 더욱 높일 수 있도록...",
  'thumbnail': 'https://image.aladin.co.kr/product/64/78/cover/8991307523_1.jpg',
  'publisher': '북타운',
  'kdc': '330.76',
  'price': '20000.0'},
 {'isbn': 9780578990385,
  'title': 'I love you more than worms love dirt (Hardcover)',
  'author': 'John Sisco (지은이)',
  'content': 'I Love You More Than... creates a happy, loving atmosphere wher

In [15]:
import collections

dict={}
test=books_df_nax['tit_cluster']

dict=collections.Counter(test)
print(dict)

Counter({88: 17529, 356: 1568, 361: 1529, 385: 1521, 136: 1433, 146: 1321, 308: 1253, 496: 1045, 281: 1022, 190: 1015, 358: 1014, 280: 1005, 400: 985, 345: 969, 442: 959, 67: 938, 374: 931, 237: 911, 61: 896, 455: 889, 328: 880, 369: 878, 178: 871, 137: 859, 256: 854, 470: 850, 419: 840, 276: 837, 349: 824, 140: 821, 104: 816, 341: 808, 451: 807, 243: 797, 188: 795, 258: 789, 260: 787, 118: 779, 397: 718, 371: 718, 372: 696, 330: 694, 445: 671, 201: 660, 27: 650, 63: 648, 388: 647, 343: 644, 382: 629, 24: 615, 443: 609, 426: 606, 334: 604, 122: 594, 181: 582, 36: 581, 130: 579, 139: 575, 399: 572, 185: 567, 427: 561, 465: 554, 234: 552, 68: 548, 376: 548, 28: 547, 450: 539, 56: 539, 103: 537, 85: 536, 286: 529, 86: 526, 74: 523, 158: 520, 108: 518, 384: 516, 22: 515, 198: 506, 3: 504, 110: 497, 161: 496, 499: 493, 38: 490, 242: 480, 300: 478, 33: 472, 25: 468, 8: 458, 296: 443, 73: 441, 327: 441, 32: 439, 90: 438, 216: 437, 309: 437, 142: 434, 11: 426, 227: 423, 305: 411, 423: 411, 270

In [ ]:

dict={}
test=books_df_nax[books_df_nax['des_cluster']==280]['content']

dict=collections.Counter(test)
print(dict)


Counter({'none': 72339, '.': 11, '아홉 살부터 고아로 가난하게 자란 은송! 돈이 최고, 돈이면 OK! 돈만 되면 뭐든지 빡세게 노동해주는 강은송(키스만 빼고 다 돼요~) 도도하고 안하무인에 다혈질적인 재벌2세 제갈원은 만인의 연인이지만 가정부 은송이만 보면 유치찬란 심술궂어 지는데 이것이 사랑인가요?': 10, '한국역사연구회의 중견 학자들이 ７년여에 걸쳐 쓴 동학혁명 연구서....': 5, '중국작가의 역사무협소설.': 5, '선화의 금월검에 베여 피를 뿌린 오성추의 몸이 힘없이 땅바닥에 처박혔다. 방금 전까지 검광과 살육이 난무하며 사람과 사람이 뒤엉켜 아귀지옥과 다름없던 저잣거리의 한복판은 어느새 쥐죽은 듯 고요했다. 악전고투를 펼치던 금룡상단의 표사들과 태천상단의 표사들도 지금 이 순간만큼은 선화와 땅바닥에...': 5, '鬪 魂좌절하기엔 내 나이 너무 젊다.스트라이커의 숙명이 부르는 그라운드에다시금 내 혼을 불사른다.왼발은 엄청난 스핀킥!!오른발은 공포의 캐넌킥!!': 5, '2인자와의 대결이 끝나면 은퇴후 동물을 키우며 살아가려던 극도의 동물애호가인 프로레슬러 시바타 겐조가 경기 도중에 이세계로 소환당한다. 그를 소환한 공주는 왕국에 피해를 끼치는 마수를 퇴치해달라고 부탁하는데, 그 말에 분노한 겐조는 공주에게 레슬링 필살기를 날리고 순식간에 쫓기는 몸에 되어버렸는데...': 5, '영어회화교재. 기본영어를 터득한 이들이 보면 적당한 회화교재이다.': 4, '제...': 4, '기독교 성가집. 교회에서 부르는 성가곡 모음집이다.': 4, '『펀드 투자권유 대행인』은 금융투자전문인력 표준교재이다.': 4, '고도(古都) 아이테리아의 뒷골목으로 이어진 일본식 주점 ‘노부’. 게으름뱅이 위병, 제멋대로인 자작가문의 영애, 엄격한 징세청부인 등, 개성적인 이세계 주민들은 낯선 일본식 주점 요리에 입맛을 다시며, 차례차례 그 맛의 포로가 되어 가는데…?!': 4, "노블엔진에서 처음으로 선보이는 신장르, '매관☆MAGIC' 코미디가